In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import cv2
import zipfile

from google.colab import files
from google.colab import drive

In [2]:
max_words = 2000
max_len = 10
num_classes = 1

# Training
epochs = 10
batch_size = 64
print_batch_n = 100

In [3]:
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

In [4]:
df_test.head()

,id,tweet
0,31963,#studiolife #aislife #requires #passion #dedic...
1,31964,@user #white #supremacists want everyone to s...
2,31965,safe ways to heal your #acne!! #altwaystohe...
3,31966,is the hp and the cursed child book up for res...
4,31967,"3rd #bihday to my amazing, hilarious #nephew..."


In [5]:
df_train['label'].value_counts()

0    29720
1     2242
Name: label, dtype: int64

In [6]:
!pip install stop-words pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

In [8]:
sw = set(get_stop_words("en"))
# sw

In [9]:
puncts = set(punctuation)
# puncts

In [10]:
morpher = MorphAnalyzer()


def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in puncts)
    txt = txt.lower()
    txt = re.sub("не\s", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

In [11]:
df_train['tweet'].iloc[:1].values

array([' @user when a father is dysfunctional and is so selfish he drags his kids into his dysfunction.   #run'],
      dtype=object)

In [12]:
df_train['tweet'].iloc[:1].apply(preprocess_text).values

array(['user father dysfunctional selfish drags kids dysfunction run'],
      dtype=object)

In [13]:
from tqdm import tqdm 
tqdm.pandas()

df_train['tweet'] = df_train['tweet'].progress_apply(preprocess_text)
df_test['tweet'] = df_test['tweet'].progress_apply(preprocess_text)

100%|██████████| 17197/17197 [00:10<00:00, 1709.64it/s]


In [14]:
train_corpus = " ".join(df_train["tweet"])
train_corpus = train_corpus.lower()

In [15]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")

tokens = word_tokenize(train_corpus)
tokens[:50]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


['user',
 'father',
 'dysfunctional',
 'selfish',
 'drags',
 'kids',
 'dysfunction',
 'run',
 'user',
 'user',
 'thanks',
 'lyft',
 'credit',
 'cant',
 'use',
 'cause',
 'dont',
 'offer',
 'wheelchair',
 'vans',
 'pdx',
 'disapointed',
 'getthanked',
 'bihday',
 'majesty',
 'model',
 'love',
 'u',
 'take',
 'u',
 'time',
 'urð\x9f\x93±',
 'ð\x9f\x98\x99ð\x9f\x98\x8eð\x9f\x91\x84ð\x9f\x91',
 'ð\x9f\x92¦ð\x9f\x92¦ð\x9f\x92¦',
 'factsguide',
 'society',
 'now',
 'motivation',
 '22',
 'huge',
 'fan',
 'fare',
 'big',
 'talking',
 'leave',
 'chaos',
 'pay',
 'disputes',
 'get',
 'allshowandnogo']

In [16]:
tokens_filtered = [word for word in tokens if word.isalnum()]

In [17]:
from nltk.probability import FreqDist

dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]  # вычитание 1 для padding
len(tokens_filtered_top)

1999

In [18]:
tokens_filtered_top[:10]

['user', 'love', 'day', 'happy', 'amp', 'just', 'will', 'im', 'u', 'time']

In [19]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}
vocabulary

{'user': 1,
 'love': 2,
 'day': 3,
 'happy': 4,
 'amp': 5,
 'just': 6,
 'will': 7,
 'im': 8,
 'u': 9,
 'time': 10,
 'life': 11,
 'like': 12,
 'today': 13,
 'new': 14,
 'now': 15,
 'positive': 16,
 'thankful': 17,
 'get': 18,
 'people': 19,
 'good': 20,
 'bihday': 21,
 'cant': 22,
 'one': 23,
 'see': 24,
 'can': 25,
 'dont': 26,
 'fathers': 27,
 'smile': 28,
 'go': 29,
 'want': 30,
 'take': 31,
 'work': 32,
 'healthy': 33,
 'got': 34,
 'weekend': 35,
 'fun': 36,
 'make': 37,
 'summer': 38,
 'best': 39,
 'friday': 40,
 'us': 41,
 'need': 42,
 'way': 43,
 'great': 44,
 'bull': 45,
 'family': 46,
 'beautiful': 47,
 '2': 48,
 'going': 49,
 'friends': 50,
 'days': 51,
 'first': 52,
 'really': 53,
 'wait': 54,
 'music': 55,
 'morning': 56,
 'back': 57,
 'know': 58,
 'world': 59,
 'tomorrow': 60,
 'fathersday': 61,
 'never': 62,
 'sad': 63,
 'week': 64,
 'cute': 65,
 'orlando': 66,
 'think': 67,
 'feel': 68,
 'blog': 69,
 'happiness': 70,
 'well': 71,
 'model': 72,
 'right': 73,
 'much': 74,
 

In [20]:
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])

    padding = [0] * (maxlen-len(result))
    return result[-maxlen:] + padding

In [21]:
%%time
x_train = np.asarray([text_to_sequence(text, max_len) for text in df_train["tweet"]])


CPU times: user 5.27 s, sys: 22.8 ms, total: 5.29 s
Wall time: 5.92 s


In [22]:
x_train.shape

(31962, 10)

In [23]:
df_train['tweet'].iloc[0]

'user father dysfunctional selfish drags kids dysfunction run'

In [24]:
x_train[0]

array([  1, 172, 165, 364,   0,   0,   0,   0,   0,   0])

In [25]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self, vocab_size=2000, embedding_dim=128, out_channel=128, num_classes=1):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.conv_1 = nn.Conv1d(embedding_dim, out_channel, kernel_size=2)
        self.conv_2 = nn.Conv1d(embedding_dim, out_channel, kernel_size=3)
        self.pool = nn.MaxPool1d(2)
        self.relu = nn.ReLU()
        self.linear_1 = nn.Linear(out_channel, out_channel // 2)
        self.linear_2 = nn.Linear(out_channel // 2, num_classes)
        
    def forward(self, x):        
        output = self.embedding(x) # B, L, E
        #                       B  E  L         
        output = output.permute(0, 2, 1)
        output = self.conv_1(output)
        output = self.relu(output)
        output = self.pool(output)

        output = self.conv_2(output)
        output = self.relu(output)
        output = self.pool(output)
        output = torch.max(output, axis=2).values
        output = self.linear_1(output)
        output = self.relu(output)
        output = self.linear_2(output)
        output = F.sigmoid(output)
        return output

In [26]:
from torch.utils.data import DataLoader, Dataset


class DataWrapper(Dataset):
    def __init__(self, data, target, transform=None):
        self.data = torch.from_numpy(data).long()
        self.target = torch.from_numpy(target).long()
        self.transform = transform
        
    def __getitem__(self, index):
        x = self.data[index]
        y = self.target[index]
        
        if self.transform:
            x = self.transform(x)
            
        return x, y
    
    def __len__(self):
        return len(self.data)

In [27]:
df_train['tweet'].shape

(31962,)

In [28]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x_train, df_train['label'].values, 
                                                    test_size=0.3)

train_dataset = DataWrapper(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_dataset = DataWrapper(X_test, y_test)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=True)

In [29]:
for x, l in train_loader:
    print(x.shape)
    print(l.shape)
    print(l[0])
    break

torch.Size([64, 10])
torch.Size([64])
tensor(0)


In [30]:
model = Net(vocab_size=max_words)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [31]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.BCELoss()

In [32]:
model = model.to(device)
model.train()
th = 0.5

train_loss_history = []
test_loss_history = []


for epoch in range(epochs):  
    running_items, running_right = 0.0, 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        
        # обнуляем градиент
        optimizer.zero_grad()
        outputs = model(inputs)
        
        loss = criterion(outputs, labels.float().view(-1, 1))
        loss.backward()
        optimizer.step()

        # подсчет ошибки на обучении
        loss = loss.item()
        running_items += len(labels)
        # подсчет метрики на обучении
        pred_labels = torch.squeeze((outputs > th).int())
        running_right += (labels == pred_labels).sum()
        
    # выводим статистику о процессе обучения
    model.eval()
    
    print(f'Epoch [{epoch + 1}/{epochs}]. ' \
            f'Step [{i + 1}/{len(train_loader)}]. ' \
            f'Loss: {loss:.3f}. ' \
            f'Acc: {running_right / running_items:.3f}', end='. ')
    running_loss, running_items, running_right = 0.0, 0.0, 0.0
    train_loss_history.append(loss)

        # выводим статистику на тестовых данных
    test_running_right, test_running_total, test_loss = 0.0, 0.0, 0.0
    for j, data in enumerate(val_loader):
        test_labels = data[1].to(device)
        test_outputs = model(data[0].to(device))
        
        # подсчет ошибки на тесте
        test_loss = criterion(test_outputs, test_labels.float().view(-1, 1))
        # подсчет метрики на тесте
        test_running_total += len(data[1])
        pred_test_labels = torch.squeeze((test_outputs > th).int())
        test_running_right += (test_labels == pred_test_labels).sum()
    
    test_loss_history.append(test_loss.item())
    print(f'Test loss: {test_loss:.3f}. Test acc: {test_running_right / test_running_total:.3f}')
    
    model.train()
        
print('Training is finished!')

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1944: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Epoch [1/10]. Step [350/350]. Loss: 0.119. Acc: 0.935. Test loss: 0.795. Test acc: 0.941
Epoch [2/10]. Step [350/350]. Loss: 0.087. Acc: 0.955. Test loss: 0.023. Test acc: 0.949
Epoch [3/10]. Step [350/350]. Loss: 0.107. Acc: 0.962. Test loss: 0.043. Test acc: 0.948
Epoch [4/10]. Step [350/350]. Loss: 0.153. Acc: 0.969. Test loss: 0.088. Test acc: 0.951
Epoch [5/10]. Step [350/350]. Loss: 0.030. Acc: 0.978. Test loss: 0.130. Test acc: 0.941
Epoch [6/10]. Step [350/350]. Loss: 0.016. Acc: 0.982. Test loss: 0.068. Test acc: 0.947
Epoch [7/10]. Step [350/350]. Loss: 0.009. Acc: 0.984. Test loss: 0.000. Test acc: 0.946
Epoch [8/10]. Step [350/350]. Loss: 0.021. Acc: 0.987. Test loss: 0.000. Test acc: 0.945
Epoch [9/10]. Step [350/350]. Loss: 0.125. Acc: 0.987. Test loss: 0.000. Test acc: 0.943
Epoch [10/10]. Step [350/350]. Loss: 0.038. Acc: 0.989. Test loss: 0.000. Test acc: 0.943
Training is finished!


# Parameters
epochs = 5

batch_size = 512

max_words = 2000

max_len = 20

# Result
Epoch [1/5]. Step [44/44]. Loss: 0.220. Acc: 0.931. Test loss: 0.114. Test acc: 0.926

Epoch [2/5]. Step [44/44]. Loss: 0.174. Acc: 0.935. Test loss: 0.026. Test acc: 0.939

Epoch [3/5]. Step [44/44]. Loss: 0.117. Acc: 0.955. Test loss: 0.002. Test acc: 0.944

Epoch [4/5]. Step [44/44]. Loss: 0.097. Acc: 0.968. Test loss: 0.648. Test acc: 0.942

Epoch [5/5]. Step [44/44]. Loss: 0.069. Acc: 0.976. Test loss: 0.839. Test acc: 0.939

# Parameters
epochs = 5

batch_size = 128

max_words = 2000

max_len = 10

# Result
Epoch [1/5]. Step [175/175]. Loss: 0.176. Acc: 0.936. Test loss: 0.756. Test acc: 0.943

Epoch [2/5]. Step [175/175]. Loss: 0.127. Acc: 0.955. Test loss: 0.599. Test acc: 0.947

Epoch [3/5]. Step [175/175]. Loss: 0.039. Acc: 0.965. Test loss: 0.001. Test acc: 0.946

Epoch [4/5]. Step [175/175]. Loss: 0.091. Acc: 0.974. Test loss: 0.127. Test acc: 0.945

Epoch [5/5]. Step [175/175]. Loss: 0.044. Acc: 0.981. Test loss: 0.000. Test acc: 0.947

# Parameters
epochs = 10

batch_size = 64

max_words = 2000

max_len = 10

# Result
Epoch [1/10]. Step [350/350]. Loss: 0.119. Acc: 0.935. Test loss: 0.795. Test acc: 0.941

Epoch [2/10]. Step [350/350]. Loss: 0.087. Acc: 0.955. Test loss: 0.023. Test acc: 0.949

Epoch [3/10]. Step [350/350]. Loss: 0.107. Acc: 0.962. Test loss: 0.043. Test acc: 0.948

Epoch [4/10]. Step [350/350]. Loss: 0.153. Acc: 0.969. Test loss: 0.088. Test acc: 0.951

Epoch [5/10]. Step [350/350]. Loss: 0.030. Acc: 0.978. Test loss: 0.130. Test acc: 0.941

Epoch [6/10]. Step [350/350]. Loss: 0.016. Acc: 0.982. Test loss: 0.068. Test acc: 0.947

Epoch [7/10]. Step [350/350]. Loss: 0.009. Acc: 0.984. Test loss: 0.000. Test acc: 0.946

Epoch [8/10]. Step [350/350]. Loss: 0.021. Acc: 0.987. Test loss: 0.000. Test acc: 0.945

Epoch [9/10]. Step [350/350]. Loss: 0.125. Acc: 0.987. Test loss: 0.000. Test acc: 0.943

Epoch [10/10]. Step [350/350]. Loss: 0.038. Acc: 0.989. Test loss: 0.000. Test acc: 0.943

In [32]:
Заметил, что при уменьшении batch_size и max_len можно добиться незначительного улучшения качества, но так как изначально метрика и так неплохая

# Заметил, что при уменьшении batch_size и max_len можно добиться незначительного улучшения качества, но, возможно это потому что изначально метрика неплохая. Возможно прирост был бы больше на, изначально, плохой метрике. Увеличение количествоа Эпох ведет также небольшому улучшению. 